<a href="https://colab.research.google.com/github/leech2193/weather_hospital_sowhat/blob/main/20220714_18_eseul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 2-2 날씨 빅데이터 콘테스트 unzip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_to_zip_file = '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트.zip'
directory_to_extract_to = '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트'

import zipfile

with zipfile.ZipFile(path_to_zip_file, 'r') as zf:
	zipinfo = zf.infolist()
	for info in zipinfo:
		info.filename = info.filename.encode('cp437').decode('euc-kr')
		zf.extract(info, directory_to_extract_to)

### 예보데이터 전처리 작업 코드 작성 (날짜 및 지역 반영)

In [2]:
import pandas as pd
import numpy as np

In [ ]:
path = '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/2012/강원/갈말읍_일최저기온_201201_201212.csv'
df = pd.read_csv(path, header=None)
df.columns = ['day','hour','forecast', 'value']
df.columns

Index(['day', 'hour', 'forecast', 'value'], dtype='object')

In [ ]:
df.loc[0, 'yyyymmdd'] = df.loc[0,'value'].strip()[-8:]
df.loc[df['day'].str.strip().str.startswith('Start'), 'yyyymmdd'] = df.loc[df['day'].str.strip().str.startswith('Start'), 'day'].str.strip().str[-8:]
df.yyyymmdd = df.yyyymmdd.fillna(method='ffill')
df.yyyymmdd = pd.to_datetime(df.yyyymmdd)
df['yyyy']     = df.yyyymmdd.dt.year
df['mm']       = df.yyyymmdd.dt.month
df['area'] = path.split('/')[-2]
df.rename(columns = {'value' : path.split('/')[-1].split('_')[-3]}, inplace=True)

drop_idx = df[(df['hour'].isna()) & (df['forecast'].isna())].index
df = df.drop(drop_idx).drop(0)

In [ ]:
df.head()

,day,hour,forecast,일최저기온,yyyymmdd,yyyy,mm,area
1,1,0200,+4,-13.900000,2012-01-01,2012,1,강원
2,1,0200,+7,-9.900000,2012-01-01,2012,1,강원
3,1,0500,+4,-13.900000,2012-01-01,2012,1,강원
4,1,0500,+7,-9.900000,2012-01-01,2012,1,강원
5,1,0800,+4,-12.900000,2012-01-01,2012,1,강원


In [ ]:
df.loc[494:505,:]

,day,hour,forecast,일최저기온,yyyymmdd,yyyy,mm,area
494,31,2000,+7,-19.900000,2012-01-01,2012,1,강원
495,31,2300,+4,-21.900000,2012-01-01,2012,1,강원
496,31,2300,+7,-19.900000,2012-01-01,2012,1,강원
498,1,0200,+4,-21.900000,2012-02-01,2012,2,강원
499,1,0200,+7,-19.900000,2012-02-01,2012,2,강원
500,1,0500,+4,-21.900000,2012-02-01,2012,2,강원
501,1,0500,+7,-19.900000,2012-02-01,2012,2,강원
502,1,0800,+4,-22.900000,2012-02-01,2012,2,강원
503,1,0800,+7,-19.900000,2012-02-01,2012,2,강원
504,1,1100,+4,-22.900000,2012-02-01,2012,2,강원


In [ ]:
import os
os.listdir('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트')

['2012', '2013', '2014', '2015', '2016']

In [ ]:
import glob
glob.glob('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/*.csv', recursive=True)

[]

In [ ]:
file_lst = glob.glob('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/*/*/*.csv')
len(file_lst)
file_lst

In [ ]:
file_lst[0].split('/')[-1].split('_')[-3]

'12시간강수량'

### 데이터 중복 제거

In [ ]:
path = '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/2013/강원/갈말읍_습도_201301_201312.csv'
dup_df = pd.read_csv(path, header=None)
dup_df.columns = ['day','hour','forecast', 'value']
# df.columns

dup_df.loc[0, 'yyyymmdd'] = dup_df.loc[0,'value'].strip()[-8:]
dup_df.loc[dup_df['day'].str.strip().str.startswith('Start'), 'yyyymmdd'] = dup_df.loc[dup_df['day'].str.strip().str.startswith('Start'), 'day'].str.strip().str[-8:]
dup_df.yyyymmdd = dup_df.yyyymmdd.fillna(method='ffill')
dup_df.yyyymmdd = pd.to_datetime(dup_df.yyyymmdd)
dup_df['yyyy']     = dup_df.yyyymmdd.dt.year
dup_df['mm']       = dup_df.yyyymmdd.dt.month
dup_df['area'] = path.split('/')[-2]
dup_df.rename(columns = {'value' : path.split('/')[-1].split('_')[-3]}, inplace=True)

drop_idx = dup_df[(dup_df['hour'].isna()) & (dup_df['forecast'].isna())].index
dup_df = dup_df.drop(drop_idx).drop(0)

dup_df.head()


,day,hour,forecast,습도,yyyymmdd,yyyy,mm,area
1,1,0200,+4,75.000000,2013-01-01,2013,1,강원
2,1,0200,+7,80.000000,2013-01-01,2013,1,강원
3,1,0200,+10,90.000000,2013-01-01,2013,1,강원
4,1,0200,+13,90.000000,2013-01-01,2013,1,강원
5,1,0200,+16,90.000000,2013-01-01,2013,1,강원


In [ ]:
dup_df.index[dup_df['day'].str.contains('format')][0]

48191

In [ ]:
drop_dup_df = dup_df.iloc[dup_df.index < dup_df.index[dup_df['day'].str.contains('format')][0]]
dup_df.shape, drop_dup_df.shape

((96361, 8), (48180, 8))

### male, female concat

In [ ]:
female = pd.read_csv('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/hospital_female_0721.csv', encoding='cp949', index_col=0)
male = pd.read_csv('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/hospital_male_0721.csv', encoding='cp949', index_col=0)
female.shape, male.shape

((24837, 28), (24837, 28))

In [ ]:
display(female.head())
display(male.head())

,area,tma,yyyy,mm,dd,avg_ta,max_ta,min_ta,sum_gsr,sum_rn,...,pm10,sex,frequency,elderly_pop,total_pop,elderly_ratio,avg_age,rhm_min_max_dif,rhm_min_avg_dif,D/R
0,강원,2012-01-01,2012.0,1.0,1.0,-4.226027,0.140278,-9.140278,0.000000,0.173611,...,86.279167,2,3,137678.0,762617.0,18.053361,42.4,46.177143,23.088571,3.933823
1,경기,2012-01-01,2012.0,1.0,1.0,-3.962264,0.322642,-8.922642,0.000000,0.037736,...,91.567357,2,5,631291.0,5928150.0,10.649039,37.7,50.131034,25.065517,0.843433
2,경남,2012-01-01,2012.0,1.0,1.0,0.335897,5.789744,-4.284615,0.000000,0.065789,...,59.554398,2,1,247889.0,1644251.0,15.076105,40.5,38.266667,19.133333,0.608180
3,경북,2012-01-01,2012.0,1.0,1.0,-0.666667,4.298246,-5.329825,0.000702,0.721053,...,76.618056,2,1,258405.0,1342455.0,19.248690,43.2,34.010526,17.005263,0.744904
4,광주,2012-01-01,2012.0,1.0,1.0,-0.200000,2.880000,-3.620000,0.000000,0.700000,...,44.745833,2,2,81546.0,738784.0,11.037868,37.6,30.800000,15.400000,2.707151


,area,tma,yyyy,mm,dd,avg_ta,max_ta,min_ta,sum_gsr,sum_rn,...,pm10,sex,frequency,elderly_pop,total_pop,elderly_ratio,avg_age,rhm_min_max_dif,rhm_min_avg_dif,D/R
0,강원,2012-01-01,2012.0,1.0,1.0,-4.226027,0.140278,-9.140278,0.000000,0.173611,...,86.279167,1,3,96281.0,772718.0,12.460044,39.4,46.177143,23.088571,3.882400
1,경기,2012-01-01,2012.0,1.0,1.0,-3.962264,0.322642,-8.922642,0.000000,0.037736,...,91.567357,1,4,447450.0,6020446.0,7.432174,36.1,50.131034,25.065517,0.664403
2,경남,2012-01-01,2012.0,1.0,1.0,0.335897,5.789744,-4.284615,0.000000,0.065789,...,59.554398,1,2,152866.0,1665308.0,9.179443,37.4,38.266667,19.133333,1.200979
3,경북,2012-01-01,2012.0,1.0,1.0,-0.666667,4.298246,-5.329825,0.000702,0.721053,...,76.618056,1,6,167115.0,1354649.0,12.336406,39.5,34.010526,17.005263,4.429192
4,광주,2012-01-01,2012.0,1.0,1.0,-0.200000,2.880000,-3.620000,0.000000,0.700000,...,44.745833,1,0,55792.0,726529.0,7.679253,35.3,30.800000,15.400000,0.000000


In [ ]:
# unnamed 삭제
female.drop(columns = 'Unnamed: 0.1', inplace=True)
male.drop(columns = 'Unnamed: 0.1', inplace=True)

In [ ]:
gender_lform = pd.concat([female, male])
gender_lform = gender_lform.sort_values(by=['area', 'tma'])
gender_lform.yyyy = gender_lform.yyyy.astype(int)
gender_lform.mm   = gender_lform.mm.astype(int)
gender_lform.dd   = gender_lform.dd.astype(int)
tmp = gender_lform[['sex','frequency']]
gender_lform.drop(['sex','frequency'], axis = 1, inplace=True)
gender_lform = pd.concat([gender_lform, tmp], axis = 1)
gender_lform = gender_lform.reset_index(drop=True)
gender_lform.head()

,area,tma,yyyy,mm,dd,avg_ta,max_ta,min_ta,sum_gsr,sum_rn,...,pm10,elderly_pop,total_pop,elderly_ratio,avg_age,rhm_min_max_dif,rhm_min_avg_dif,D/R,sex,frequency
0,강원,2012-01-01,2012,1,1,-4.226027,0.140278,-9.140278,0.0,0.173611,...,86.279167,137678.0,762617.0,18.053361,42.4,46.177143,23.088571,3.933823,2,3
1,강원,2012-01-01,2012,1,1,-4.226027,0.140278,-9.140278,0.0,0.173611,...,86.279167,96281.0,772718.0,12.460044,39.4,46.177143,23.088571,3.882400,1,3
2,강원,2012-01-02,2012,1,2,-6.504110,-0.559722,-11.276389,0.0,0.050000,...,70.744444,137678.0,762617.0,18.053361,42.4,45.771429,22.885714,0.000000,2,0
3,강원,2012-01-02,2012,1,2,-6.504110,-0.559722,-11.276389,0.0,0.050000,...,70.744444,96281.0,772718.0,12.460044,39.4,45.771429,22.885714,2.588266,1,2
4,강원,2012-01-03,2012,1,3,-6.215068,-0.566667,-11.344444,0.0,0.027778,...,87.895833,137678.0,762617.0,18.053361,42.4,48.108571,24.054286,0.000000,2,0


In [ ]:
gender_lform.to_csv('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/hospital_gender_longform_0725.csv', encoding='cp949', index=False)

### 파생변수 추가
- 요일
- 계절
- workday or holiday

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/hospital_gender_longform.csv', encoding='cp949')
df.head(3)

,area,tma,yyyy,mm,dd,avg_ta,max_ta,min_ta,sum_gsr,sum_rn,...,min_ps,so2,o3,pm10,elderly_pop,total_pop,elderly_ratio,avg_age,sex,frequency
0,강원,2012-01-01,2012,1,1,-4.226027,0.140278,-9.140278,0.0,0.173611,...,1020.193333,0.011685,0.026750,91.145833,137678.0,762617.0,18.053361,42.4,2,3
1,강원,2012-01-01,2012,1,1,-4.226027,0.140278,-9.140278,0.0,0.173611,...,1020.193333,0.011685,0.026750,91.145833,96281.0,772718.0,12.460044,39.4,1,3
2,강원,2012-01-02,2012,1,2,-6.504110,-0.559722,-11.276389,0.0,0.050000,...,1023.960000,0.008881,0.021179,82.200397,137678.0,762617.0,18.053361,42.4,2,0


In [ ]:
# 요일 (0 : mon, 1: tue)
df.insert(5,'weekday',pd.to_datetime(df.tma).dt.dayofweek)

In [ ]:
# 계절
def season(frm) :
    spring, summer, autumn, winter = [3,4,5], [6,7,8], [9,10,11], [12,1,2]
    if frm in spring :
        return 0
    elif frm in summer :
        return 1
    elif frm in autumn :
        return 2
    elif frm in winter :
        return 3

df.insert(6,'season', df.mm.apply(season))

In [ ]:
df.head(3)

,area,tma,yyyy,mm,dd,weekday,season,avg_ta,max_ta,min_ta,...,min_ps,so2,o3,pm10,elderly_pop,total_pop,elderly_ratio,avg_age,sex,frequency
0,강원,2012-01-01,2012,1,1,6,3,-4.226027,0.140278,-9.140278,...,1020.193333,0.011685,0.026750,91.145833,137678.0,762617.0,18.053361,42.4,2,3
1,강원,2012-01-01,2012,1,1,6,3,-4.226027,0.140278,-9.140278,...,1020.193333,0.011685,0.026750,91.145833,96281.0,772718.0,12.460044,39.4,1,3
2,강원,2012-01-02,2012,1,2,0,3,-6.504110,-0.559722,-11.276389,...,1023.960000,0.008881,0.021179,82.200397,137678.0,762617.0,18.053361,42.4,2,0


### 파일에 start 12갠지 check..

In [3]:
import glob
file_lst = glob.glob('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/*/*/*.csv')
print(len(file_lst))
file_lst[:5]

5892


['/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/2012/강원/갈말읍_12시간강수량_201201_201212.csv',
 '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/2012/강원/갈말읍_12시간신적설_201201_201212.csv',
 '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/2012/강원/갈말읍_3시간기온_201201_201212.csv',
 '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/2012/강원/갈말읍_강수형태_201201_201212.csv',
 '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/2012/강원/갈말읍_강수확률_201201_201212.csv']

In [4]:
# 숫자만 count해보기 > df 저장
chk_cnt = pd.DataFrame(columns = ['year','area','area1','data','start_cnt', 'format_cnt', 'path'])

for chk_file in file_lst :
    test_file  = pd.read_csv(chk_file, encoding='cp949', header=None)
    format_cnt = test_file[0].str.contains('format').sum()
    start_cnt  = test_file[0].str.contains('Start').sum() + format_cnt
    year  = chk_file.split('/')[-3]
    area  = chk_file.split('/')[-2]
    area1 = chk_file.split('/')[-1].split('_')[0]
    data  = chk_file.split('/')[-1].split('_')[1]

    new_data = [year, area, area1, data, start_cnt, format_cnt, chk_file]
    chk_cnt.loc[len(chk_cnt)] = new_data

    # else :
    #     print(chk_file)
    # print('start  count : ', start_cnt)
    # print('format count : ', format_cnt)
    # print('*' * 50)

In [27]:
print('abnormal...T.T')
print('start  count (not 12) : ', (chk_cnt.start_cnt != 12).sum()) # start + format (중복인 경우 double)
print('format count (not  1) : ', (chk_cnt.format_cnt != 1).sum())

abnormal...T.T
start  count (not 12) :  2950
format count (not  1) :  1246


In [20]:
chk_cnt.loc[:,'normal'] = 'normal'
chk_cnt.loc[(chk_cnt.start_cnt != 12) | (chk_cnt.format_cnt != 1), 'normal'] = 'abnormal'

chk_cnt.head(3)

,year,area,area1,data,start_cnt,format_cnt,path,normal
0,2012,강원,갈말읍,12시간강수량,12,1,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,normal
1,2012,강원,갈말읍,12시간신적설,12,1,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,normal
2,2012,강원,갈말읍,3시간기온,10,1,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,abnormal


In [7]:
import unicodedata
for i in range(chk_cnt.shape[0]) :
    for j in range(chk_cnt.shape[1]) :
        chk_cnt.iloc[i,j] = unicodedata.normalize('NFC', str(chk_cnt.iloc[i,j]))
print('complete')

complete


In [19]:
chk_cnt.year = chk_cnt.year.astype('int')
chk_cnt.start_cnt = chk_cnt.start_cnt.astype('int')
chk_cnt.format_cnt = chk_cnt.format_cnt.astype('int')

In [22]:
# csv 파일로 빼기
chk_cnt.to_csv('/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/file_chk.csv', encoding='cp949', index=False)

In [26]:
chk_cnt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5892 entries, 0 to 5891
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        5892 non-null   int64 
 1   area        5892 non-null   object
 2   area1       5892 non-null   object
 3   data        5892 non-null   object
 4   start_cnt   5892 non-null   int64 
 5   format_cnt  5892 non-null   int64 
 6   path        5892 non-null   object
 7   normal      5892 non-null   object
dtypes: int64(3), object(5)
memory usage: 414.3+ KB


In [59]:
abnormal = chk_cnt[chk_cnt.normal == 'abnormal']
print('2012 total    - ', chk_cnt[chk_cnt.year == 2012].start_cnt.unique())
print('2012 abnormal - ', abnormal[abnormal.year == 2012].start_cnt.unique(), '\n')

print('2013 total    - ', chk_cnt[chk_cnt.year == 2013].start_cnt.unique())
print('2013 abnormal - ', abnormal[abnormal.year == 2013].start_cnt.unique(), '\n')

print('2014 total    - ', chk_cnt[chk_cnt.year == 2014].start_cnt.unique())
print('2014 abnormal - ', abnormal[abnormal.year == 2014].start_cnt.unique(), '\n')

print('2015 total    - ', chk_cnt[chk_cnt.year == 2015].start_cnt.unique())
print('2015 abnormal - ', abnormal[abnormal.year == 2015].start_cnt.unique(), '\n')

print('2016 total    - ', chk_cnt[chk_cnt.year == 2016].start_cnt.unique())
print('2016 abnormal - ', abnormal[abnormal.year == 2016].start_cnt.unique())

2012 total    -  [12 10  1  5  9]
2012 abnormal -  [10  1  5  9] 

2013 total    -  [12 24  8  1 22]
2013 abnormal -  [24  8  1 22] 

2014 total    -  [12 11  1]
2014 abnormal -  [11  1] 

2015 total    -  [12  1 24]
2015 abnormal -  [ 1 24] 

2016 total    -  [12  1 11  3 24 22  6 48 44]
2016 abnormal -  [ 1 11  3 24 22  6 48 44]


In [60]:
chk_cnt.data.unique()

array(['12시간강수량', '12시간신적설', '3시간기온', '강수형태', '강수확률', '습도', '일최고기온',
       '일최저기온', '파고', '풍속', '풍향', '하늘상태', '6시간강수량', '6시간적설'], dtype=object)

In [ ]:
chk_cnt[(chk_cnt.data == '습도') & (chk_cnt.format_cnt == '2')]

,year,area,area1,data,start_cnt,format_cnt,path,normal
1183,2013,강원,갈말읍,습도,22,2,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,abnormal
1197,2013,강원,대관령면,습도,22,2,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,abnormal
1211,2013,강원,명륜1동,습도,22,2,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,abnormal
1225,2013,강원,사천면,습도,22,2,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,abnormal
1239,2013,강원,송정동,습도,22,2,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,abnormal
...,...,...,...,...,...,...,...,...
5837,2016,충북,교현.안림동,습도,22,2,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,abnormal
5849,2016,충북,보은읍,습도,22,2,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,abnormal
5861,2016,충북,복대1동,습도,22,2,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,abnormal
5873,2016,충북,용두동,습도,22,2,/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data...,abnormal


In [ ]:
path_ = '/content/drive/MyDrive/공모전/날씨빅데이터콘테스트(공유)/data/2-2날씨빅데이터 콘테스트/2014/충남/부여읍_습도_201401_201412.csv'
df_ = pd.read_csv(path_, header=None)
df_.columns = ['day','hour','forecast', 'value']
df_.columns

Index(['day', 'hour', 'forecast', 'value'], dtype='object')

In [ ]:
for i in range(0, len(df)):
                df.loc[i, 'yyyymmdd2'] = df.loc[i, 'yyyymmdd'] + datetime.timedelta(hours=test.loc[i][0]) - datetime.timedelta(days=1)

In [ ]:
df_.iloc[4093].isna()

day         False
hour         True
forecast     True
value        True
Name: 4093, dtype: bool

In [ ]:
from datetime import datetime, timedelta

pd.DatetimeIndex(df['tma']) + timedelta(hours=26)

DatetimeIndex(['2012-01-02 02:00:00', '2012-01-02 02:00:00',
               '2012-01-03 02:00:00', '2012-01-03 02:00:00',
               '2012-01-04 02:00:00', '2012-01-04 02:00:00',
               '2012-01-05 02:00:00', '2012-01-05 02:00:00',
               '2012-01-06 02:00:00', '2012-01-06 02:00:00',
               ...
               '2015-12-28 02:00:00', '2015-12-28 02:00:00',
               '2015-12-29 02:00:00', '2015-12-29 02:00:00',
               '2015-12-30 02:00:00', '2015-12-30 02:00:00',
               '2015-12-31 02:00:00', '2015-12-31 02:00:00',
               '2016-01-01 02:00:00', '2016-01-01 02:00:00'],
              dtype='datetime64[ns]', name='tma', length=49674, freq=None)